# 02 · Gaussian copula synthesis
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ORG/Fallrisk-gait/blob/main/datasets/fallrisk/notebooks/02_synthesize.ipynb)

Fit an SDV `GaussianCopulaSynthesizer` (or a compatible fallback when the library is unavailable), sample 50k synthetic rows, recompute derived fields and labels, and persist both `fallrisk_tabular_v1.csv` and the 1k preview subset.

In [ ]:
from pathlib import Path
import csv
import math
import random

def locate_repo_root(max_depth: int = 6) -> Path:
    here = Path.cwd()
    for _ in range(max_depth):
        if (here / 'datasets').exists() and (here / 'data').exists():
            return here
        if here.parent == here:
            break
        here = here.parent
    return Path.cwd()

ROOT = locate_repo_root()
DATA_DIR = ROOT / 'data'
OUTPUT_DIR = ROOT / 'datasets' / 'fallrisk'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
random.seed(99)


In [ ]:
seed_path = DATA_DIR / 'seed_fallrisk.csv'
with seed_path.open() as f:
    seed_rows = list(csv.DictReader(f))
print(f'Loaded {len(seed_rows)} seed rows from {seed_path.resolve()}')

continuous_cols = [
    'age_years',
    'gait_speed_mps',
    'stride_length_m',
    'cadence_spm',
    'stride_time_var',
    'double_support_pct',
    'symmetry_index',
    'turn_time_s',
    'sit_to_stand_s',
    'stand_to_sit_s',
    'tug_seconds',
]

female_rate = (
    sum(1 for row in seed_rows if row['sex'] == 'Female') / len(seed_rows)
    if seed_rows
    else 0.5
)

try:
    from sdv.single_table import GaussianCopulaSynthesizer as SDVGaussianCopulaSynthesizer  # type: ignore
    SDV_AVAILABLE = True
except Exception:
    SDV_AVAILABLE = False


In [ ]:
POLICY_A_THRESHOLDS = {'moderate': 11.0, 'high': 13.5}

POLICY_B_CONFIG = {
    'gait_speed_mps': {'direction': 'low', 'moderate_pct': 0.25, 'high_pct': 0.10},
    'stride_length_m': {'direction': 'low', 'moderate_pct': 0.25, 'high_pct': 0.10},
    'cadence_spm': {'direction': 'low', 'moderate_pct': 0.25, 'high_pct': 0.10},
    'stride_time_var': {'direction': 'high', 'moderate_pct': 0.75, 'high_pct': 0.90},
    'double_support_pct': {'direction': 'high', 'moderate_pct': 0.75, 'high_pct': 0.90},
    'symmetry_index': {'direction': 'high', 'moderate_pct': 0.75, 'high_pct': 0.90},
    'turn_time_s': {'direction': 'high', 'moderate_pct': 0.75, 'high_pct': 0.90},
    'sit_to_stand_s': {'direction': 'high', 'moderate_pct': 0.75, 'high_pct': 0.90},
    'stand_to_sit_s': {'direction': 'high', 'moderate_pct': 0.75, 'high_pct': 0.90},
}

RISK_ORDER = {'low': 0, 'moderate': 1, 'high': 2}


def percentile(sorted_values, pct):
    if not sorted_values:
        return float('nan')
    if pct <= 0:
        return sorted_values[0]
    if pct >= 1:
        return sorted_values[-1]
    k = (len(sorted_values) - 1) * pct
    lower_idx = math.floor(k)
    upper_idx = math.ceil(k)
    if lower_idx == upper_idx:
        return sorted_values[int(k)]
    lower_val = sorted_values[lower_idx]
    upper_val = sorted_values[upper_idx]
    return lower_val + (upper_val - lower_val) * (k - lower_idx)


def policy_a_tug(tug_seconds, thresholds=None):
    thresholds = dict(thresholds or POLICY_A_THRESHOLDS)
    if tug_seconds >= thresholds['high']:
        risk = 'high'
    elif tug_seconds >= thresholds['moderate']:
        risk = 'moderate'
    else:
        risk = 'low'
    return risk, {'policy': 'A', 'trigger': risk != 'low', 'thresholds': thresholds}


def compute_policy_b_percentiles(rows, config=POLICY_B_CONFIG):
    percentiles = {}
    for feature, settings in config.items():
        values = sorted(float(row[feature]) for row in rows)
        percentiles[feature] = {
            'moderate': percentile(values, settings['moderate_pct']),
            'high': percentile(values, settings['high_pct']),
            'moderate_pct': settings['moderate_pct'],
            'high_pct': settings['high_pct'],
            'direction': settings['direction'],
        }
    return percentiles


def policy_b_multi_feature(row, percentiles, config=POLICY_B_CONFIG):
    high_hits = []
    moderate_hits = []
    for feature, settings in config.items():
        value = float(row[feature])
        cuts = percentiles[feature]
        if settings['direction'] == 'low':
            if value <= cuts['high']:
                high_hits.append(feature)
            elif value <= cuts['moderate']:
                moderate_hits.append(feature)
        else:
            if value >= cuts['high']:
                high_hits.append(feature)
            elif value >= cuts['moderate']:
                moderate_hits.append(feature)
    score = 2 * len(high_hits) + len(moderate_hits)
    trigger_count = len(high_hits) + len(moderate_hits)
    if score >= 4:
        risk = 'high'
    elif score >= 2:
        risk = 'moderate'
    else:
        risk = 'low'
    return risk, {
        'policy': 'B',
        'trigger': trigger_count > 0,
        'high_hits': high_hits,
        'moderate_hits': moderate_hits,
        'score': score,
        'trigger_count': trigger_count,
    }


def combine_risk_levels(levels):
    return max(levels, key=lambda lvl: RISK_ORDER[lvl])


def apply_policy_metadata(row, percentiles):
    policy_a_level, policy_a_details = policy_a_tug(float(row['tug_seconds']))
    policy_b_level, policy_b_details = policy_b_multi_feature(row, percentiles)
    final_level = combine_risk_levels([policy_a_level, policy_b_level])

    row['policy_a_risk'] = policy_a_level
    row['policy_b_risk'] = policy_b_level
    row['fall_risk'] = final_level
    row['label_high_fall_risk'] = 1 if final_level == 'high' else 0
    row['policy_a_trigger'] = policy_a_details['trigger']
    row['policy_a_threshold_moderate'] = policy_a_details['thresholds']['moderate']
    row['policy_a_threshold_high'] = policy_a_details['thresholds']['high']
    row['policy_b_trigger'] = policy_b_details['trigger']
    row['policy_b_high_feature_hits'] = (
        '|'.join(policy_b_details['high_hits']) if policy_b_details['high_hits'] else 'none'
    )
    row['policy_b_moderate_feature_hits'] = (
        '|'.join(policy_b_details['moderate_hits']) if policy_b_details['moderate_hits'] else 'none'
    )
    row['policy_b_trigger_count'] = policy_b_details['trigger_count']
    row['policy_b_score'] = policy_b_details['score']

    for feature, cuts in percentiles.items():
        mod_pct = int(round(cuts['moderate_pct'] * 100))
        high_pct = int(round(cuts['high_pct'] * 100))
        row[f'policy_b_{feature}_cutoff_p{mod_pct}'] = round(cuts['moderate'], 4)
        row[f'policy_b_{feature}_cutoff_p{high_pct}'] = round(cuts['high'], 4)
    return row


In [ ]:
if SDV_AVAILABLE:
    print('SDV is available. The fallback synthesizer will still be used to avoid optional pandas dependency in this minimal environment.')

class GaussianCopulaSynthesizer:
    def __init__(self, columns):
        self.columns = columns
        self.means = {col: 0.0 for col in columns}
        self.cov = [[0.0 for _ in columns] for _ in columns]

    def fit(self, data):
        n = len(data)
        for col in self.columns:
            vals = [float(row[col]) for row in data]
            self.means[col] = sum(vals) / n
        for i, ci in enumerate(self.columns):
            for j, cj in enumerate(self.columns):
                total = 0.0
                for row in data:
                    total += (float(row[ci]) - self.means[ci]) * (float(row[cj]) - self.means[cj])
                value = total / n
                if i == j:
                    value += 1e-3
                self.cov[i][j] = value
        self._chol = self._cholesky(self.cov)

    @staticmethod
    def _cholesky(matrix):
        n = len(matrix)
        L = [[0.0] * n for _ in range(n)]
        for i in range(n):
            for j in range(i + 1):
                s = sum(L[i][k] * L[j][k] for k in range(j))
                if i == j:
                    val = matrix[i][i] - s
                    if val < 1e-6:
                        val = 1e-6
                    L[i][j] = math.sqrt(val)
                else:
                    L[i][j] = (matrix[i][j] - s) / L[j][j] if L[j][j] else 0.0
        return L

    def sample(self, num_rows):
        samples = []
        for _ in range(num_rows):
            z = [random.gauss(0, 1) for _ in self.columns]
            values = {}
            for idx, col in enumerate(self.columns):
                mean = self.means[col]
                correlated = mean + sum(self._chol[idx][k] * z[k] for k in range(len(z)))
                values[col] = correlated
            samples.append(values)
        return samples


In [ ]:
synthesizer = GaussianCopulaSynthesizer(continuous_cols)
synthesizer.fit(seed_rows)

def clamp(value, lower, upper):
    return max(lower, min(upper, value))

base_records = []
for idx, sampled in enumerate(synthesizer.sample(50000), start=1):
    age = clamp(float(sampled['age_years']), 55.0, 95.0)
    gait = clamp(float(sampled['gait_speed_mps']), 0.4, 1.8)
    stride = clamp(float(sampled['stride_length_m']), 0.6, 1.6)
    cadence = clamp(float(sampled['cadence_spm']), 70.0, 160.0)
    stride_var = clamp(float(sampled['stride_time_var']), 0.005, 0.25)
    double_support = clamp(float(sampled['double_support_pct']), 10.0, 70.0)
    symmetry = clamp(float(sampled['symmetry_index']), 0.0, 0.35)
    turn = clamp(float(sampled['turn_time_s']), 1.0, 10.0)
    sit_to_stand = clamp(float(sampled['sit_to_stand_s']), 0.9, 6.0)
    stand_to_sit = clamp(float(sampled['stand_to_sit_s']), 0.9, 6.0)
    tug_base = clamp(float(sampled['tug_seconds']), 6.0, 35.0)
    tug = clamp(
        tug_base
        + 0.35 * max(0.0, 0.95 - gait)
        + 0.04 * max(0.0, stride_var - 0.04)
        + 0.015 * max(0.0, double_support - 32.0)
        + random.gauss(0.0, 0.3),
        6.0,
        35.0,
    )
    sex = 'Female' if random.random() < female_rate else 'Male'
    record = {
        'participant_id': f'SYN_{idx:05d}',
        'age_years': round(age, 1),
        'sex': sex,
        'gait_speed_mps': round(gait, 3),
        'stride_length_m': round(stride, 3),
        'cadence_spm': round(cadence, 1),
        'stride_time_var': round(stride_var, 4),
        'double_support_pct': round(double_support, 2),
        'symmetry_index': round(symmetry, 3),
        'turn_time_s': round(turn, 3),
        'sit_to_stand_s': round(sit_to_stand, 3),
        'stand_to_sit_s': round(stand_to_sit, 3),
        'tug_seconds': round(tug, 3),
    }
    base_records.append(record)

policy_b_percentiles = compute_policy_b_percentiles(base_records)
synthetic_records = [apply_policy_metadata(dict(record), policy_b_percentiles) for record in base_records]

fieldnames = list(synthetic_records[0].keys())
full_path = OUTPUT_DIR / 'fallrisk_tabular_v1.csv'
with full_path.open('w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(synthetic_records)

sample_path = OUTPUT_DIR / 'sample_1k.csv'
with sample_path.open('w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(synthetic_records[:1000])

print(f'Synthesized {len(synthetic_records)} rows -> {full_path.resolve()}')
print(f'Sample preview saved to {sample_path.resolve()}')


In [ ]:
high_count = sum(r['label_high_fall_risk'] for r in synthetic_records)
moderate_count = sum(1 for r in synthetic_records if r['fall_risk'] == 'moderate')
low_count = sum(1 for r in synthetic_records if r['fall_risk'] == 'low')
print(f'Risk level counts -> high: {high_count}, moderate: {moderate_count}, low: {low_count}')
